# 1. 누락 데이터 처리

머신 러닝 등 데이터 분석의 정확도는 분석 데터의 품질에 의해 좌우된다.<br>
데이터 품질을 높이기 위해서는 누락 데이터, 중복 데이터 등 오류를 수정하고 분석 목적에 맞게 변형하는 과정이 필요하다.<br>
수집한 데이터를 분석에 적합하도록 사전 처리(Preprocessing)하는 방법을 살펴보자.
<br><br>
데이터프레임에는 원소 데이터 값이 종종 누락되는 경우가 있다.<br>
데이터를 파일로 입력할 때 빠트리거나 파일 형식을 변환하는 과정에서 데이터가 소실되는 것이 주요 원인이다.<br>
일반적으로 유효한 데이터 값이 존재하지 않는 누락 데이터를 NaN(Not a Number)으로 표시한다.
<br><br>
머신러닝 분석 모형에 데이터를 입력하기 전에 반드시 누락 데이터를 제거하거나 다른 적절한 값으로 대체하는 과정이 필요하다.<br>
누락 데이터가 많아지면 데이터의 품질이 떨어지고, 머신러닝 분석 알고리즘을 왜곡하는 현상이 발생하기 때문이다.

### - 누락 데이터 확인

Seaborn 라이브러리의 'titanic' 데이터셋을 사용한다.<br>
`head()` 메소드로 첫 5행을 출력하면 'deck' 열에 NaN 값이 있다. 이 승객의 경우 몇 번 데크에 승선했는지 데이터가 없다는 뜻이다.

In [6]:
import seaborn as sns

# titanic 데이터셋 가져오기
df = sns.load_dataset('titanic')
print(df.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  


---

`info()` 메소드로 데이터프레임의 요약 정보를 출력하면 각 열에 속하는 데이터 중에서 유효한(non-null, 즉 NaN 값이 아닌) 값의 개수를 보여준다.<br>
RangeIndex를 보면 각 열에 891개의 데이터가 있다. 그리고 'deck' 열에는 203개의 유효한 범주형 데이터가 있다.<br>
따라서 'deck' 열에 있는 누락 데이터가 688개라는 사실을 알 수 있다.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


---

`value_counts()` 메소드를 이용하여 'deck' 열에 688개의 누락 데이터가 있는 것을 확인할 수도 있다.<br>
단, 이때 누락 데이터의 개수를 확인하려면 반드시 `dropna=False` 옵션을 사용한다.<br>
그렇지 않으면 NaN 값을 제외하고 유효한 데이터의 개수만을 구하기 때문이다.

In [8]:
# deck 열의 NaN 개수 계산하기
nan_deck = df['deck'].value_counts(dropna=False)
print(nan_deck)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64


---

누락 데이터를 찾는 직접적인 방법으로 `isnull()` 메소드와 `notnull()` 메소드가 있다.

- isnull(): 누락 데이터면 True를 반환하고, 유효한 데이터가 존재하면 False를 반환한다.
- notnull(): 유효한 데이터가 존재하면 True를 반환하고, 누락 데이터면 False를 반환한다.

먼저 데이터프레임에 `head()` 메소드를 적용하면 첫 5행으로 구성된 데이터프레임 객체를 반환한다.<br>
예제에서는 첫 5행의 원소들이 누락 데이터인지 여부를 `isnull()` 메소드를 적용하여 판별한다.<br>
예를 들면, 'deck' 열의 0행에 있는 원소는 True 값이므로 누락 데이터이다.

In [10]:
# isnull() 메소드로 누락 데이터 찾기
print(df.head().isnull())

   survived  pclass    sex    age  sibsp  parch   fare  embarked  class  \
0     False   False  False  False  False  False  False     False  False   
1     False   False  False  False  False  False  False     False  False   
2     False   False  False  False  False  False  False     False  False   
3     False   False  False  False  False  False  False     False  False   
4     False   False  False  False  False  False  False     False  False   

     who  adult_male   deck  embark_town  alive  alone  
0  False       False   True        False  False  False  
1  False       False  False        False  False  False  
2  False       False   True        False  False  False  
3  False       False  False        False  False  False  
4  False       False   True        False  False  False  


---

`notnull()` 메소드를 적용하면 유효한 값이 있는 경우 True를 반환하고 누락 데이터가 있는 경우 False를 반환한다.<br>
'deck' 열의 0행에 위치한 원소는 False 값을 갖기 때문에 누락 데이터이다.

In [11]:
# notnull() 메소드로 누락 데이터 찾기
print(df.head().notnull())

   survived  pclass   sex   age  sibsp  parch  fare  embarked  class   who  \
0      True    True  True  True   True   True  True      True   True  True   
1      True    True  True  True   True   True  True      True   True  True   
2      True    True  True  True   True   True  True      True   True  True   
3      True    True  True  True   True   True  True      True   True  True   
4      True    True  True  True   True   True  True      True   True  True   

   adult_male   deck  embark_town  alive  alone  
0        True  False         True   True   True  
1        True   True         True   True   True  
2        True  False         True   True   True  
3        True   True         True   True   True  
4        True  False         True   True   True  


---

누락 데이터의 개수를 구할 때 `isnull()` 메소드와 `notnull()` 메소드를 활용할 수 있다.<br>
`isnull()` 메소드의 경우 반환되는 값이 참이면 1이고 거짓이면 0으로 판별한다.<br>
따라서 `isnull()` 메소드를 실행하고 sum(axis=0) 메소드를 적용하면 참(1)의 합을 구한다.<br>
이처럼 각 열의 누락 데이터(NaN) 개수를 구할 수 있다.<br>
계산 결과를 보면, 'deck' 열에만 3개의 누락 데이터가 존재한다.


In [12]:
# isnull() 메소드로 누락 데이터 개수 구하기
print(df.head().isnull().sum(axis=0))


survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           3
embark_town    0
alive          0
alone          0
dtype: int64
